In [3]:
import pandas as pd

# File paths for the uploaded files
train_scores_path = 'train_scores.csv'
train_logs_path = 'train_logs.csv'
sample_submission_path = 'sample_submission.csv'
test_logs_path = 'test_logs.csv'

# Loading the data into Pandas DataFrames
train_scores = pd.read_csv(train_scores_path)
train_logs = pd.read_csv(train_logs_path)
sample_submission = pd.read_csv(sample_submission_path)
test_logs = pd.read_csv(test_logs_path)

# Display the first few rows of each DataFrame to understand their structure
train_scores.head(), train_logs.head(), sample_submission.head(), test_logs.head()

(         id  score
 0  001519c8    3.5
 1  0022f953    3.5
 2  0042269b    6.0
 3  0059420b    2.0
 4  0075873a    4.0,
          id  event_id  down_time  up_time  action_time       activity  \
 0  001519c8         1       4526     4557           31  Nonproduction   
 1  001519c8         2       4558     4962          404  Nonproduction   
 2  001519c8         3     106571   106571            0  Nonproduction   
 3  001519c8         4     106686   106777           91          Input   
 4  001519c8         5     107196   107323          127          Input   
 
   down_event   up_event text_change  cursor_position  word_count  
 0  Leftclick  Leftclick    NoChange                0           0  
 1  Leftclick  Leftclick    NoChange                0           0  
 2      Shift      Shift    NoChange                0           0  
 3          q          q           q                1           1  
 4          q          q           q                2           1  ,
          id  score
 0  